In [2]:
import sys
import os
import logging
import time
import numpy as np
from argparse import ArgumentParser
from IPython.config.loader import PyFileConfigLoader


import PyDragonfly
from PyDragonfly import CMessage, MT_EXIT, copy_to_msg, copy_from_msg
from dragonfly_utils import respond_to_ping

import Dragonfly_config as rc
import quaternionarray as qa
import amcmorl_py_tools.vecgeom as vg
import PolarisDragonfly as pd

C:\Anaconda\lib\site-packages\IPython\config.py:13: ShimWarning: The `IPython.config` package has been deprecated. You should import from traitlets.config instead.
  "You should import from traitlets.config instead.", ShimWarning)


In [2]:
vg?

In [61]:
positions = np.zeros([200,4])
orientations = np.zeros([200,5])
np.testing.assert_array_equal(positions[:,0], orientations[:,0], err_msg='Samples are not aligned')

In [2]:
'''

'''

class HotspotLocator (object):
    def __init__(self, config_file, mm_ip):
        self.load_config(config_file)
        self.load_logging()
        self.setup_dragonfly(mm_ip)
        self.user_start_calibrate()
        self.run()
        
    def load_config(self, config_file):
        cfg = PyFileConfigLoader(config_file)
        cfg.load_config()
        self.config = cfg.config
        
        # special casing for SAMPLE_GENERATED
        if (self.config.trigger == 'SAMPLE_GENERATED'):
            self.config.trigger_msg = rc.MT_SAMPLE_GENERATED
            self.config.trigger_mdf = rc.MDF_SAMPLE_GENERATED
        else:
            self.config.trigger_msg = \
                eval('rc.MT_' + self.config.trigger)
            self.config.trigager_mdf = \
                eval('rc.MDF_' + self.config.trigger)
        print "Triggering with", self.config.trigger
        print "PolarisDragonfly: loading config"
        
        self.ntools = len(self.config.tool_list)
           
    def load_logging(self):
        log_file = os.path.normpath(os.path.join(self.config.config_dir, 'coil_calibration.log'))
        print "log file: " + log_file
        logging.basicConfig(filename=log_file, level=logging.DEBUG)
        logging.info(' ')
        logging.info(' ')
        logging.debug("**** STARTING UP ****")
        logging.info("  %s  " % time.asctime())
        logging.info("*********************")
            
    def setup_dragonfly(self, mm_ip):
        self.mod = PyDragonfly.Dragonfly_Module(0, 0)
        self.mod.ConnectToMMM(mm_ip)
        self.mod.Subscribe(MT_EXIT)
        self.mod.Subscribe(rc.MT_PING)
        self.mod.Subscribe(self.config.trigger_msg)
        self.mod.Subscribe(rc.MT_POLARIS_POSITION)
        
        self.mod.SendModuleReady()
        print "PolarisDragonfly: connected to dragonfly"
           
    def user_start_calibrate(self):
        while True:
            x = raw_input("Press enter to calibrate...")
            if not x:
                break
            print '.......'
        sys.stdout.write('starting in:')
        sys.stdout.write('3\n')
        sys.stdout.flush()
        time.sleep(1)
        sys.stdout.write('2\n')
        sys.stdout.flush()
        time.sleep(1)
        sys.stdout.write('1\n')
        sys.stdout.flush()
        time.sleep(1)
        sys.stdout.write('Calibrating...')
        self.create_storage()
    
    def create_storage (self):
        self.store_pos = np.empty([time_to_collect * fsamp_per_chan * self.ntools, 4])
        self.store_ori = np.empty([time_to_collect * fsamp_per_chan * self.ntools, 5])
        self.store_i = 0
        self.calibrated = False

    def run(self):
        while True:
            msg = CMessage()
            rcv = self.mod.ReadMessage(msg, 0.001)
            if rcv == 1:
                msg_type = msg.GetHeader().msg_type
                dest_mod_id = msg.GetHeader().dest_mod_id
                if  msg_type == MT_EXIT:
                    if (dest_mod_id == 0) or (dest_mod_id == self.mod.GetModuleID()):
                        print 'Received MT_EXIT, disconnecting...'
                        self.mod.SendSignal(rc.MT_EXIT_ACK)
                        self.mod.DisconnectFromMMM()
                        break;
                elif msg_type == rc.MT_PING:
                    respond_to_ping(self.mod, msg, 'PolarisDragonfly')
                else:
                    self.process_msg(msg)
    
    def process_message(self, in_msg):

        msg_type = in_msg.GetHeader().msg_type
        if msg_type == rc.MT_POLARIS_POSITION:
            # handling input message
            in_mdf = rc.MDF_POLARIS_POSITION()
            copy_from_msg(in_mdf, in_msg)
            positions = in_mdf.xyz[:] 
            orientations =in_mdf.ori[:]            
            
            #need to shuffle quarternion somewhere to x,y,z, w
            # possibly storing bias values
            if self.calibrated:
                 # calculating output
                self.Qk = orientations[self.config.tools.index('CT315')::2, 1:] #need to find a way to discriminate the tools files in the messages???
                Qr = qa.mult(sample, qa.inv(self.Qk))
                Tk = positions[self.config.tools.index('CT315')::2, 1:] 
                hotspot_position[n, :] = qa.rotate(Qr, self.Xi) + Tk

                #creating output message
                out_mdf = rc.MDF_HOTSPOT_POSITION()
                out_mdf.xyz[:] = hotspot_position
                out_mdf.ori[:] = 0 # Qk - coil active orientation
                out_mdf.sample_header = in_mdf.sample_header
                msg = CMessage(rc.MT_HOTSPOT_POSITION)
                copy_to_msg(out_mdf, msg)
                self.mod.SendMessage(msg)
                sys.stdout.write("o")
            else:
                self.store_pos[self.store_i, 1:] = positions
                self.store_ori[self.store_i, 1:] = orientations # Why is it this way?
                self.store_i += 1
                if (self.store_i >= self.store_pos.shape[0]) & (self.store_i >= self.store_ori.shape[0]):
                    self.calibrating = False
                    self.make_calibration_vector()
            
    def make_calibration_vector(self):
        plate_ori = self.store_ori[self.config.tools.index('CB609')::2].mean(axis=0)
        Ni = self.store_pos[self.config.tools.index('CB609')::2].mean(axis=0)
        self.Qi = self.store_ori[self.config.tools.index('CT315')::2].mean(axis=0)
        Ti = self.store_pos[self.config.tools.index('CT315')::2].mean(axis=0)
        msg_str_pos = "%.5e, " * 3
        msg_str_ori = "%.5e, " * 4
        sys.stdout.write('Plate orientation:    ')
        sys.stdout.write(msg_str_ori % (plate_ori[0], plate_ori[1], plate_ori[2],\
                                    plate_ori[3]) + "\n")
        sys.stdout.write('Plate position:       ')
        sys.stdout.write(msg_str_pos % (Ni[0], Ni[1], Ni[2]) + "\n")
        sys.stdout.write('Coil orientation:     ')
        sys.stdout.write(msg_str_ori % (self.Qi[0], self.Qi[1], self.Qi[2], self.Qi[3]) + "\n")
        sys.stdout.write('Coil position:        ')
        sys.stdout.write(msg_str_pos % (Ti[0], Ti[1], Ti[2]) + "\n")
        self.Xi = Ni - Ti
        sys.stdout.write('Vector:        ')
        sys.stdout.write(msg_str_pos % (self.Xi[0], self.Xi[1], self.Xi[2]) + "\n")
        sys.stdout.write("********** Calibration complete! ***********\n")
        sys.stdout.flush()
        self.calibrated = True

if __name__ == "__main__":
    parser = ArgumentParser(description = 'Interface with Polaris hardware' \
        ' and emit POLARIS_POSITION messages')
    parser.add_argument(type=str, dest='config')
    parser.add_argument(type=str, dest='mm_ip', nargs='?', default='')
    args = parser.parse_args()
    print("Using config file=%s, MM IP=%s" % (args.config, args.mm_ip))
    pdf = PolarisDragonfly(args.config, args.mm_ip)
    print "Finishing up"

usage: __main__.py [-h] config [mm_ip]
__main__.py: error: unrecognized arguments: -f


SystemExit: 2

To exit: use 'exit', 'quit', or Ctrl-D.


Changelog:

26/11/2015
* OMEGADF- change classname to coil_calibration
* Include load_logging method from PolarisDragonfly
* unbias_ft becomes create_storage
* setup_dragonfly uses mm_ip instead of server
* Add user_start_calibrate method
* rezero method renamed vector
* vector outputs mean of orientations and positions for coil and calibration plate
* vector prints calculated vector
* vector and coil_ori become objects
* process_message recieves POLARIS_POSITION
* process_message creates positions and orientation arrays
* process_message removes sample number from array
* create_storage is called from User_start_calibrate
* Replaces load_config method with PolarisDragonfly corresponding method
* Added config 'main' information to load_config... ~~**Do we need a n_tool variable?**~~ **Does the polaris output a list of tools is can currently see?**
* Log file saves to coil_calibration log file
* setup_polaris_interface not called as method currently
* **setup_dragonfly may need subscriptions added for Polaris_Position?**
* Removed UR5 movement messages and commands


27/11/2015
* Added a calculation in process_message to find the hospot position for each sample **Probably doesn't work curently**
* Changed vector to make_calibration_vector
* Now subscribing to POLARIS_POSITION
* Reorganized to improve readability
* Renaming of some variables to improve readability... ongoing


30/11/2015
* Added ntools data member
* Simplified calculations loops in process_message


* *What needs changing in process_message after #calculating outputs?*
* *What adjustments need to be made to get_ft_vals_from_DAQ_DATA_buffer?*
* *What is \__name\__ == "\__main\__" doing in this instance, does anything need changing?*
* *Do we need a seperate function to do calculation or should this sit within process message to occur if unbias = False*

In [2]:
store_coil_ori = np.empty([200, 4])
calib_coil_ori = np.empty([4])
coil_position = np.empty([200, 3])
vector = np.empty([3])

In [22]:
def vector_calculation(store_coil_ori, calib_coil_ori, coil_position, vector):
    active_coil_ori = store_coil_ori[::2] #need to find a way to discriminate the tools files in the messages???
    coil_rotation = np.empty(active_coil_ori.shape) 
    for n in range(len(active_coil_ori)):
        coil_rotation[n, :] = qa.mult(active_coil_ori[n], qa.inv(calib_coil_ori))
    print coil_rotation
    coil_position = coil_position[::2]
    hotspot_position = np.empty(coil_position.shape)
    for n in range(len(active_coil_ori)):
        hotspot_position[n, :] = qa.rotate(coil_rotation[n], vector) + coil_position[n]
    return hotspot_position

In [4]:
def load_config(config_file):
        cfg = PyFileConfigLoader(config_file)
        cfg.load_config()
        config = cfg.config
        return config

In [5]:
config = load_config('C:\\Users\\amcmorl\\Documents\\BCI\\app_configs\\realTMSmap\\polaris_dragonfly_config.py')

In [6]:
config.tools

['CT315', 'CB609']

In [23]:
x = vector_calculation(store_coil_ori, calib_coil_ori, coil_position, vector)

[[  0.00000000e+000   0.00000000e+000   0.00000000e+000   0.00000000e+000]
 [ -1.81537095e-142  -1.26996910e-142  -1.11717589e-142   9.27170572e-143]
 [  1.28697309e-142  -1.83967749e-142   9.39584733e-143   1.13213409e-142]
 [ -1.44259962e-224   1.19724738e-224   2.34417289e-224   1.63990017e-224]
 [  0.00000000e+000   0.00000000e+000   0.00000000e+000   0.00000000e+000]
 [ -3.73175205e-121  -2.61060131e-121  -2.29651325e-121   1.90593040e-121]
 [ -7.08280649e-027  -4.95488009e-027  -4.35874589e-027   3.61742582e-027]
 [ -1.69933753e-094   1.41032021e-094   2.76136282e-094   1.93175145e-094]
 [  0.00000000e+000   0.00000000e+000   0.00000000e+000   0.00000000e+000]
 [  0.00000000e+000   0.00000000e+000   0.00000000e+000   0.00000000e+000]
 [  0.00000000e+000   0.00000000e+000   0.00000000e+000   0.00000000e+000]
 [ -9.40503841e-136   7.80546276e-136   1.52828517e-135   1.06913407e-135]
 [  0.00000000e+000   0.00000000e+000   0.00000000e+000   0.00000000e+000]
 [  5.29148190e-103   6.3

In [2]:
def user_start_calibrate():
        while True:
            x = raw_input("Press enter to calibrate...")
            if not x:
                break
            print '.......'
        sys.stdout.write('starting in:\n')
        sys.stdout.write('3\n')
        time.sleep(1)
        sys.stdout.write('2\n')
        time.sleep(1)
        sys.stdout.write('1\n')
        time.sleep(1)
        sys.stdout.write('Calibrating...')

In [1]:
mm_ip = 'localhost:7111'
mod = PyDragonfly.Dragonfly_Module(0, 0)
mod.ConnectToMMM(mm_ip)
mod.Subscribe(rc.MT_POLARIS_POSITION)
mod.SendModuleReady()

NameError: name 'PyDragonfly' is not defined

In [3]:
user_start_calibrate()

Press enter to calibrate...
starting in:
3
2
1
Calibrating...

In [42]:
ni = np.empty([200, 3])
Qi = np.empty([200, 4])
def vector(store_ori, store_pos, start_time, firq):
    start_collect = start_time * firq
    plate_ori_bias = store_ori[start_collect::2].mean(axis=0)
    plate_pos_bias = store_pos[start_collect::2].mean(axis=0)
    coil_ori_bias = store_ori[start_collect+1::2].mean(axis=0)
    coil_pos_bias = store_pos[start_collect+1::2].mean(axis=0)
    msg_str_pos = "%.5e, " * 3
    msg_str_ori = "%.5e, " * 4
    sys.stdout.write('Plate orientation:    ')
    sys.stdout.write(msg_str_ori % (plate_ori_bias[0], plate_ori_bias[1], plate_ori_bias[2],\
                                plate_ori_bias[3]) + "\n")
    sys.stdout.write('Plate position:       ')
    sys.stdout.write(msg_str_pos % (plate_pos_bias[0], plate_pos_bias[1], plate_pos_bias[2]) + "\n")
    sys.stdout.write('Coil orientation:     ')
    sys.stdout.write(msg_str_ori % (coil_ori_bias[0], coil_ori_bias[1], coil_ori_bias[2], \
                                coil_ori_bias[3]) + "\n")
    sys.stdout.write('Coil position:        ')
    sys.stdout.write(msg_str_pos % (coil_pos_bias[0], coil_pos_bias[1], coil_pos_bias[2]) + "\n")
    vector = plate_pos_bias - coil_pos_bias
    sys.stdout.write('Vector:        ')
    sys.stdout.write(msg_str_pos % (vector[0], vector[1], vector[2]) + "\n")
    sys.stdout.write("********** Calibration complete! ***********\n")
    sys.stdout.flush()

In [8]:
Qi[0::2].mean(axis=0)[0]

6.5054001993171015e-307

In [43]:
vector(Qi, ni, 0, 50)

Plate orientation:    1.65831e-307, 1.77123e-307, 1.79766e-307, 1.50227e-307, 
Plate position:       1.05947e+275, 7.25155e+269, 1.05947e+275, 
Coil orientation:     1.70198e-307, 1.69058e-307, 1.65580e-307, 1.70142e-307, 
Coil position:        1.42660e+251, 1.22751e+264, 7.25155e+269, 
Vector:        1.05947e+275, 7.25154e+269, 1.05946e+275, 
********** Calibration complete! ***********


In [13]:
coil_calibration.user_start_calibrate(2)

NameError: name 'coil_calibration' is not defined

In [33]:
msg = PyDragonfly.CMessage()
rcv = -1
tool = 1
tool_id = 0
while rcv != 1 and tool_id != tool:
    rcv = mod.ReadMessage(msg, 0.01)
    if rcv == 1:
        header = msg.GetHeader()
        if header.msg_type == rc.MT_POLARIS_POSITION:
            mdf = rc.MDF_POLARIS_POSITION()
            PyDragonfly.copy_from_msg(mdf, msg)
            tool_id = mdf.tool_id
            if tool_id == tool:
                print(np.asarray(mdf.xyz[:]))
                print(np.asarray(mdf.ori[:]))
        else:
            print("x")

[ -241.62162781  -202.62493896 -2088.58251953]
[ 0.89071101 -0.21268009 -0.17807722 -0.3601267 ]


In [3]:
Ni = np.array([-93, -101.5, -2112])
Ti = np.array([-369.904, -134.983, -2143.489])
Qi = qa.norm(np.array([-0.17681, -0.005837, -0.66479, 0.725777])) # xyz, w

In [6]:
Xi = Ni - Ti
print Xi

[ 276.904   33.483   31.489]


In [34]:
Qk = qa.norm(np.asarray([-0.21268009, -0.17807722, -0.3601267, 0.89071101]))
Tk = np.asarray([-241.62162781, -202.62493896, -2088.58251953])
Nk = np.asarray([-70.6, 16.1, -2072])
Qr = qa.mult(Qk, qa.inv(Qi)).flatten()
print Qr

[-0.11315385 -0.04633178  0.36100933  0.92451167]


In [69]:
#Qk = qa.norm(np.array([0.242, -0.023, -0.938, 0.248]))
#Tk = np.array([-261, 11, -2182])
#Nk = np.array([-231,-38,-2178])
Qr = qa.mult(Qk, qa.inv(Qi)).flatten()
print Qr

[ 0.04779975 -0.05308806 -0.44715677 -0.89159838]


In [35]:
qa.rotate(Qr, Xi) + Tk, Nk

(array([[  -65.35255198,    14.54887876, -2065.06147732]]),
 array([  -70.6,    16.1, -2072. ]))

In [55]:
Xi

array([29, 57, -4])

In [54]:
coil_ori = np.empty([200, 4])
calib_ori = np.asarray([1, 1, 1, 1])

In [56]:
for sample in coil_ori:
    for n in range(len(coil_ori)):
        coil_ori[n, :] = qa.mult(sample, qa.inv(calib_ori))

In [57]:
coil_ori

array([[ -9.76868387e-187,  -7.75708946e-187,  -1.07734936e-187,
         -1.58012796e-187],
       [ -9.76868387e-187,  -7.75708946e-187,  -1.07734936e-187,
         -1.58012796e-187],
       [ -9.76868387e-187,  -7.75708946e-187,  -1.07734936e-187,
         -1.58012796e-187],
       [ -9.76868387e-187,  -7.75708946e-187,  -1.07734936e-187,
         -1.58012796e-187],
       [ -9.76868387e-187,  -7.75708946e-187,  -1.07734936e-187,
         -1.58012796e-187],
       [ -9.76868387e-187,  -7.75708946e-187,  -1.07734936e-187,
         -1.58012796e-187],
       [ -9.76868387e-187,  -7.75708946e-187,  -1.07734936e-187,
         -1.58012796e-187],
       [ -9.76868387e-187,  -7.75708946e-187,  -1.07734936e-187,
         -1.58012796e-187],
       [ -9.76868387e-187,  -7.75708946e-187,  -1.07734936e-187,
         -1.58012796e-187],
       [ -9.76868387e-187,  -7.75708946e-187,  -1.07734936e-187,
         -1.58012796e-187],
       [ -9.76868387e-187,  -7.75708946e-187,  -1.07734936e-187,
     

In [1]:
Qk

NameError: name 'Qk' is not defined